In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import re

df_input = pd.read_csv('../input_data_1~273.csv')
df_input=df_input.fillna('')
df_input.iloc[102]

pl_id                                                          103
pl_maker                                                   samsung
pl_group                                                   samsung
pl_model_code                                             SHW-M585
pl_name                                                      갤럭시코어
pl_model_name                                           GalaxyCore
Unnamed: 6                                                        
링크제목                                                        갤럭시 코어
나무위키링크단품         https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9...
나무위키링크시리즈        https://namu.wiki/w/%EA%B0%A4%EB%9F%AD%EC%8B%9...
Name: 102, dtype: object

In [1]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "모델명": "",
    "모델코드": "",
    "제조회사": "",
    "출시일": "",
    "운영체제": "",
    "판매방식": "",

    "화면크기(센치)": "",
    "화면크기(인치)":"",
    "패널종류": "",
    "화면해상도": "", 
    "ppi": "", 
    "최대주사율": "",
    "화면비": "",
    "화면면적": "",
    "화면폭": "",
    "화면높이": "",
    "HDR규격": "",

    "AP종류": "", 
    "코어갯수": "",
    "코어클럭": "", 
    "그래픽코어": "",
    "NPU & DSP": "",
    "Sensor Hub": "",
    "통신모뎀": "",
    "시스템램": "",
    "내장메모리": "",
    "저장장치": "",
    "외장메모리": "",
    
    "5G": "",
    "4G": "",
    "WI-FI주파수": "",
    "블루투스": "",
    "위성항법": "",
    "유심타입": "",
    "듀얼유심": "",
    
    "카메라타입": "",
    "후면카메라": "",
    "전면카메라": "",
    "동영상촬영": "",
    "Flash": "",
    "사진촬영 해상도": "",
    "전면 해상도": "",
    "전면 동영상 해상도": "",
    "조리개 값": "",
    "카메라 특징": "",
    "손떨림보정": "",
    
    "손떨림방지": "",
    "카메라플래시": "",
    "HDR촬영지원": "",
    "오토HDR": "",
    "광학줌": "",
    "지오태그": "",
    "레이저 오토포커스": "",
    "오토 포커스": "",
    "아웃 포커스": "",
    "터치 포커스": "",
    "파노라마": "",
    "TOF센서": "",
    "야간모드 촬영": "",
    
    "이어폰단자": "",
    "스피커": "",
    "고음질재생": "",
    "사운드 기술": "",
    
    "지문인식": "",
    "얼굴인식": "",
    "홍채인식": "",
    "음성잠금해제": "",
    "AI/인공지능": "",
    "전자결제": "",
    "터치펜": "",
    "방수/방진": "",
    
    "기타": "",
    "충전단자": "",
    "배터리용량": "",
    "배터리타입": "",
    "배터리특징": "",
    "배터리장착방식": "",
    "충전지원": "",
    "고속충전기술": "",
    "무선충전": "",
    
    "색상": "",
    
    "가로": "",
    "세로": "",
    "두께": "",
    "무게": ""
    
 }
crawl_data = pd.DataFrame(my_dict,index=[0])
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################


#####################################################################################
######################### 함수 정의 ######################################
#####################################################################################
def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False
    
#####################################################################################
######################### 함수 정의 ######################################
#####################################################################################
    
    


# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('../chromedriver')
driver.implicitly_wait(3)
for index in range(238,len(df_input)):
#     try:
       
        temp_dict = dict()
        
        temp_dict["모델명"] = df_input.iloc[index][4]
        temp_dict["모델코드"] = df_input.iloc[index][3]
        temp_dict["제조회사"] = df_input.iloc[index][1]
        
        print(index,df_input.iloc[index]['나무위키링크단품'])
        if df_input.iloc[index]['나무위키링크단품'] != "":
            driver.get(df_input.iloc[index]['나무위키링크단품'])


            req = driver.page_source
            soup=BeautifulSoup(req, 'html.parser')
            table = ""
            if soup.select(".wiki-heading-content > .wiki-paragraph > div > .wiki-table-wrap > .wiki-table"):
                table = soup.select(".wiki-heading-content > .wiki-paragraph > div > .wiki-table-wrap > .wiki-table")[0]
            elif soup.select(".wiki-heading-content > .wiki-table-wrap > .wiki-table"):
                table = soup.select(".wiki-heading-content > .wiki-table-wrap > .wiki-table")[0]
            else:
                pass
                
            trs = table.select("tr")
            next_is_core_clock = 0
            for tr in trs:
                td = tr.select("td")
                if len(td) <= 0:
                    continue
                #####################################프로세서파트##################################################################
                #####################################프로세서파트##################################################################
                
                if td[0].text == "프로세서" and "AP종류" not in temp_dict:
                    processor = td[1].text
                    #temp_dict["프로세서"] = processor
                    
                    if processor.find("구성 내용 확인") != -1:
                        temp_dict["AP종류"] = re.search('.*\[ 구성 내용 확인 \]', processor).group().replace("[ 구성 내용 확인 ]","")
                    
                        proc_table = td[1].select(".wiki-table-wrap > table")
                        proc_trs = proc_table[0].select("tr")

                        for proc_tr in proc_trs:
                            proc_td = proc_tr.select("td")

                            if proc_td[0].text == "CPU":
                                if proc_td[1].text.find("Hz") != -1:
                                    temp_dict["코어클럭"] = proc_td[1].text
                                    next_is_core_clock = -1
                                else:
                                    next_is_core_clock = 1

                            elif next_is_core_clock == 1:
                                temp_dict["코어클럭"] = proc_td[0].text
                                next_is_core_clock = -1

                            elif proc_td[0].text == "GPU":
                                temp_dict["그래픽코어"] = proc_td[1].text

                            elif proc_td[0].text == "NPU & DSP":
                                temp_dict["NPU & DSP"] = proc_td[1].text

                            elif proc_td[0].text == "Sensor Hub":
                                temp_dict["Sensor Hub"] = proc_td[1].text

                            elif proc_td[0].text == "통신 모뎀":
                                temp_dict["통신 모뎀"] = proc_td[1].text
                    else:
                        #[구성내용 확인] 버튼이 없는경우 따로구현할것 
                        pass

                #####################################프로세서파트 끝##################################################################
                #####################################프로세서파트 끝##################################################################


                #####################################메모리 파트##################################################################
                #####################################메모리 파트##################################################################        
                elif td[0].text == "메모리" and "시스템램" not in temp_dict:
                    memory = td[1].text
                    #temp_dict["메모리"] = memory
                    
                    memorry_arr = td[1].text.split(',')
                    
                    if len(memorry_arr) >= 1:
                        temp_dict["시스템램"] = memorry_arr[0]
                        
                    if len(memorry_arr) >= 2:
                        in_mem = ""
                        if memorry_arr[1].find("GB") != -1:
                            in_mem = re.search('(.*GB)',memorry_arr[1]).group()
                        elif memorry_arr[1].find("MB") != -1:
                            in_mem = re.search('(.*MB)',memorry_arr[1]).group()
                        mem_dev = memorry_arr[1].replace(in_mem,"")
                        temp_dict["내장메모리"] = in_mem.replace(" GB","")
                        temp_dict["저장장치"] = mem_dev.replace(" 규격 내장 메모리","")
                        
                    if len(memorry_arr) >= 3:
                        temp_dict["외장메모리"] = memorry_arr[2]
                        

                #####################################메모리 파트 끝##################################################################
                #####################################메모리 파트 끝##################################################################

                elif td[0].text == "디스플레이" and "화면비" not in temp_dict:
                    display = td[1].text
                    #temp_dict["디스플레이"] = display
                    if display.find("세부 정보 확인")!= -1:
                        display_part1 = re.search('.*\[ 세부 정보 확인 \]', display).group().replace("[ 세부 정보 확인 ]","")
                        
                        temp_dict["화면크기(인치)"] = display_part1
                        temp_dict["화면크기(인치)"] = re.search('([0-9.]+인치)', display_part1).group() 



                        temp_dict["화면비"] = re.search('[0-9.]+:[0-9.]+(\+[0-9.?]+:[0-9.?]+)? 비율', display_part1).group().replace(" 비율","")
                        temp_dict["화면해상도"] = re.search('([0-9]+ x [0-9]+)', display_part1).group() 
                        temp_dict["ppi"] = re.sub('[\(\)]','',re.search('\([0-9.-]+ ppi\)|\([0-9]+ppi\)',"".join(display_part1)).group())

                        disp_table = td[1].select(".wiki-table-wrap > table")
                        disp_trs = disp_table[0].select("tr")


                        for disp_tr in disp_trs:
                            disp_td = disp_tr.select("td")

                            if disp_td[0].text == "패널정보":
                                temp_dict["패널정보"] = disp_td[1].text

                            elif disp_td[0].text == "부가정보":
                                plus_info = disp_td[1].text
                                if plus_info.find("Hz") != -1:
                                    temp_dict["최대주사율"] = re.search('([0-9]+ Hz)|([0-9]+Hz)', plus_info).group()
                                
                    else:
                        #세부정보확인 버튼이 없는경우 따로구현할것
                        pass

                 #####################################디스플레이 파트 끝##################################################################
                #####################################디스플레이 파트 끝##################################################################

                elif td[0].text == "네트워크" and "5G" not in temp_dict:
                    network = td[1].text
                    #temp_dict["네트워크"] = network
                    if network.find("5G") != -1:
                        temp_dict["5G"] = "5G"
                    if network.find("4G") != -1:
                        temp_dict["4G"] = "4G"
                        
                #####################################네트워크 파트 끝##################################################################
                #####################################네트워크 파트 끝##################################################################
                  
                    
                    
                elif td[0].text == "근접통신" and "블루투스" not in temp_dict:
                    proximity = td[1].text
                    #temp_dict["근접통신"] = proximity
                    
                    if proximity.find("블루투스") != -1:
                        temp_dict["블루투스"] = re.sub('(블루투스|,)',"",re.search('(블루투스 [0-9.]+)', proximity).group())
                    
                
                #####################################근접통신 파트 끝##################################################################
                #####################################근접통신 파트 끝##################################################################
                  
                
                
                elif td[0].text == "위성항법" and "위성항법" not in temp_dict:
                    satelite = td[1].text
                    temp_dict["위성항법"] = satelite
                    
                    
                #####################################위성항법 파트 끝##################################################################
                #####################################위성항법 파트 끝##################################################################
                  
                
                elif td[0].text == "카메라" and "전면카메라" not in temp_dict:
                    camera = td[1].text
                    data = td[1]
                    #temp_dict["카메라"] = camera
                    
                    if camera.find("[ 전면 카메라 정보") != -1:
                        
                        if td[1].select(".wiki-table-wrap > table"):

                            front_cam_table = td[1].select(".wiki-table-wrap > table")
                            front_cam_trs = front_cam_table[0].select("tr")

                            front_camera = ""
                            for num_tr in range(len(front_cam_trs)):
                                front_cam_td = front_cam_trs[num_tr].select("td")
                                if num_tr == 0:
                                    temp_dict["flash"] = front_cam_td[-1].text
                                    
                                for td in front_cam_td:
                                    if num_tr == 0 and td.text.find("화소") != -1:
                                        front_camera = td.text
                                    elif td.text.find("화소") != -1:
                                        front_camera += (" + ") + (td.text)
                            temp_dict["전면카메라"] = front_camera
                    
                    elif camera.find("전면") != -1:
                        
                        if td[1].text == "전면 카메라":
                            pass
                        elif td[1].text == "후면 카메라":
                            pass
                        else:
                            camera = camera.replace("\n","")                       
                            temp_dict["전면카메라"] = re.search('전면[0-9 ,만/팝업형]+화소', camera).group()
                    
    
                    if data.find("[ 후면 카메라 정보") != -1:
                        if data.select(".wiki-table-wrap > table"):

                            back_cam_table = data.select(".wiki-table-wrap > table")
                            back_cam_trs = back_cam_table[0].select("tr")

                            back_camera = ""
                            for num_tr in range(len(back_cam_trs)):
                                back_cam_td = back_cam_trs[num_tr].select("td")
                                if num_tr == 0:
                                    temp_dict["flash"] = back_cam_td[-1].text
                                    
                                for td in back_cam_td:
                                    if num_tr == 0 and td.text.find("화소") != -1:
                                        back_camera = td.text
                                    elif td.text.find("화소") != -1:
                                        back_camera += (" + ") + (td.text)
                                        
                            temp_dict["후면카메라"] = back_camera
                    elif camera.find("후면") != -1:
                        temp_dict["후면카메라"] = re.search('후면( OIS 지원 )?[0-9 ,만]+화소', camera).group()
                    
                 #######################################################################################################
                #######################################################################################################
                  
                
                
                elif td[0].text == "배터리" and "배터리용량" not in temp_dict:
                    battery = td[1].text
                    #temp_dict["배터리"] = battery
                    
                    if battery.find("[ 충전 기술 정보") != -1:
                        bat_part1 = re.search('.*\[ 충전 기술 정보', battery).group().replace("[ 충전 기술 정보","")
                        storage = re.search('[0-9 .,]+mAh', bat_part1).group()
                        temp_dict["배터리용량"] = storage
                        temp_dict["배터리타입"] = bat_part1.replace(storage,"")
                    else:
                        pass
                        
                    if td[1].select(".wiki-table-wrap > table"):

                            in_table = td[1].select(".wiki-table-wrap > table")
                            in_trs = in_table[0].select("tr")

                            for tr in in_trs:
                                td = tr.select("td")
                                if td[0].text == "유선고속충전":
                                    temp_dict["배터리특징"] = td[1].text
                    
                
                  #######################################################################################################
                #######################################################################################################
                
                elif td[0].text == "운영체제" and "운영체제" not in temp_dict:
                    os = td[1].text
                    temp_dict["운영체제"] = os
                    
                  #######################################################################################################
                #######################################################################################################
                
                elif td[0].text == "규격" and td[1].text != "한국" and td[1].text != "UMTS"and td[1].text != "크기" and td[1].text != "싱글" and td[1].text != "기본" and "가로" not in temp_dict:
                    size = td[1].text
                    #temp_dict["규격"] = size
                    xyz = re.search('[0-9.x ]+mm', size).group()
                    if xyz.find("x") != -1:
                        xyz = xyz.split("x")
                        
                    if len(xyz) > 2:
                        temp_dict["가로"] = xyz[0]
                        temp_dict["세로"] = xyz[1]
                        temp_dict["두께"] = xyz[2]
                    
                    if size.find("g") != -1:
                        temp_dict["무게"] = re.search('[0-9. ]+g', size).group()
                    elif td[2].text.find("g"):
                        temp_dict["무게"] = re.search('[0-9. ]+g', td[2].text).group()
                    
                elif td[0].text == "규격" and td[1].text == "크기" and "가로" not in temp_dict:
                    size = td[2].text
                    xyz = re.search('[0-9.x ]+mm', size).group()
                    if xyz.find("x") != -1:
                        xyz = xyz.split("x")
                        
                    if len(xyz) > 2:
                        temp_dict["가로"] = xyz[0]
                        temp_dict["세로"] = xyz[1]
                        temp_dict["두께"] = xyz[2]
                
                elif td[0].text == "규격" and td[1].text == "무게" and "무게" not in temp_dict:
                    weight = td[2].text
                    weight = re.search('[0-9. ]+g', weight).group()
                    temp_dict["무게"] = weight
                    
                elif td[0].text == "규격" and td[1].text == "싱글" and "무게" not in temp_dict:
                    size = td[2].text
                    xyz = re.search('[0-9.x ]+mm', size).group()
                    if xyz.find("x") != -1:
                        xyz = xyz.split("x")
                        
                    if len(xyz) > 2:
                        temp_dict["가로"] = xyz[0]
                        temp_dict["세로"] = xyz[1]
                        temp_dict["두께"] = xyz[2]
                    
                    weight = td[3].text
                    weight = re.search('[0-9. ]+g', weight).group()
                    temp_dict["무게"] = weight
                   
                   
                    
                    
                elif td[0].text == "규격" and td[1].text == "기본" and "가로" not in temp_dict:
                    size = td[2].text
                    #temp_dict["규격"] = size
                    xyz = re.search('[0-9.x ]+mm', size).group()
                    if xyz.find("x") != -1:
                        xyz = xyz.split("x")
                        
                    if len(xyz) > 2:
                        temp_dict["가로"] = xyz[0]
                        temp_dict["세로"] = xyz[1]
                        temp_dict["두께"] = xyz[2]
                        
                    if size.find("g") != -1:
                        temp_dict["무게"] = re.search('[0-9. ]+g', size).group()
                    elif td[3].text.find("g") != -1:
                        temp_dict["무게"] = re.search('[0-9. ]+g', td[3].text).group()
                        
                elif td[0].text == "규격" and (td[1].text == "UMST" or td[1].text == "한국" ) and "가로" not in temp_dict:
                    size = td[2].text
                    #temp_dict["규격"] = size
                    xyz = re.search('[0-9.x ]+mm', size).group()
                    if xyz.find("x") != -1:
                        xyz = xyz.split("x")
                        
                    if len(xyz) > 2:
                        temp_dict["가로"] = xyz[0]
                        temp_dict["세로"] = xyz[1]
                        temp_dict["두께"] = xyz[2]
                        
                    if size.find("g") != -1:
                        temp_dict["무게"] = re.search('[0-9. ]+g', size).group()
                    elif td[3].text.find("g") != -1:
                        temp_dict["무게"] = re.search('[0-9. ]+g', td[3].text).group()
                        
                
                  #######################################################################################################
                #######################################################################################################
                
                elif td[0].text.find("색상") != -1 and "색상" not in temp_dict :
                    color = td[1].text
                    temp_dict["색상"] = color
                    
                  #######################################################################################################
                #######################################################################################################
                
                elif td[0].text == "단자정보" and "충전단자" not in temp_dict :
                    terminal = td[1].text
                    #temp_dict["단자정보"] = terminal
                    if terminal.find("x 1") != -1:
                        temp_dict["충전단자"] = re.search('(.*x 1)', terminal).group()
                        
                    elif terminal.find("단자") != -1:
                        temp_dict["충전단자"] = re.search('(.*단자)', terminal).group()
                
                  #######################################################################################################
                #######################################################################################################
                
                elif td[0].text == "생체인식":
                    biometrics = td[1].text
                    temp_dict["생체인식"] = biometrics
                    
                  #######################################################################################################
                #######################################################################################################
                
                elif td[0].text == "기타":
                    etc = td[1].text
                    temp_dict["기타"] = etc
                    
                  #######################################################################################################
                #######################################################################################################


            crawl_data = crawl_data.append(pd.DataFrame(temp_dict,index=[0]))
        

                
                                             
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.= re.search('(전면[0-9 ,만]+화소)', camera).group()
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.
 
crawl_data



NameError: name 'pd' is not defined

In [72]:
 
crawl_data

,모델명,모델코드,제조회사,출시일,운영체제,판매방식,화면크기(센치),화면크기(인치),패널종류,화면해상도,ppi,최대주사율,화면비,화면면적,화면폭,화면높이,HDR규격,AP종류,코어갯수,코어클럭,그래픽코어,NPU & DSP,Sensor Hub,통신모뎀,시스템램,내장메모리,저장장치,외장메모리,5G,4G,WI-FI주파수,블루투스,위성항법,유심타입,듀얼유심,카메라타입,후면카메라,전면카메라,동영상촬영,Flash,사진촬영 해상도,전면 해상도,전면 동영상 해상도,조리개 값,카메라 특징,손떨림보정,손떨림방지,카메라플래시,HDR촬영지원,오토HDR,광학줌,지오태그,레이저 오토포커스,오토 포커스,아웃 포커스,터치 포커스,파노라마,TOF센서,야간모드 촬영,이어폰단자,스피커,고음질재생,사운드 기술,지문인식,얼굴인식,홍채인식,음성잠금해제,AI/인공지능,전자결제,터치펜,방수/방진,기타,충전단자,배터리용량,배터리타입,배터리특징,배터리장착방식,충전지원,고속충전기술,무선충전,색상,가로,세로,두께,무게
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,삼성 마스터폰,SM-B510,samsung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,램,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FM 라디오 지원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메탈릭 실버 (모든통신기기용 대응색상),NaN,NaN,NaN,NaN
0,삼성 마스터폰 SKT 3G,SM-B510,samsung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,램,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FM 라디오 지원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메탈릭 실버 (모든통신기기용 대응색상),NaN,NaN,NaN,NaN


In [67]:
crawl_data.to_csv('data_df_2.csv', # file path, file name
    ...: sep=',',   
    ...: na_rep='NaN',
                 encoding='utf-8-sig') 



In [ ]:
#####이런식으로 변경?
row = [row.title, row.title, row.title]
series = pd.Seires(row, index=df.columns)
df = df.append(series, ignore_index = True)